In [19]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

In [20]:
# Parametros

g = 5

# Costos

a = [3, 4.05, 4.05, 3.99, 3.88]
b = [20, 18.07, 15.55, 19.21, 26.18]
c = [100, 98.87, 104,26, 107.21, 95.31]

# Emisiones

d = [2, 3.82, 5.01, 1.10, 3.55]
e = [-5, -4.24, -2.15, -3.99, -6.88]
f = [3, 6.09, 5.69, 6.20, 5.57]

# Potencias

Pmin = [28, 90, 68, 76, 19]
Pmax = [206, 284, 189, 266, 53]


In [21]:
model = gp.Model('Generación Térmica ambiental')
# Variable de desición
potgen = model.addVars(g, vtype=GRB.CONTINUOUS, name="potgen")


In [22]:
# Restricción de potencias

p_min = model.addConstrs((potgen[type] >= Pmin[type])
                              for type in range(g))

p_max = model.addConstrs((potgen[type] <= Pmax[type])
                              for type in range(g))

In [23]:
# Satisfacer la demanda

demanda = model.addConstr(gp.quicksum(potgen[type]  for type in range(g)) >= 400)

In [24]:
# Minimizar costos

costos = gp.quicksum(a[type]*potgen[type]*potgen[type]+b[type]*potgen[type]+c[type]
                    for type in range(g))

emisiones = gp.quicksum(d[type]*potgen[type]*potgen[type]+e[type]*potgen[type]+f[type]
                   for type in range(g))
 
#Restricción de emisiones

r_emisones = model.addConstr(emisiones <= 90000)


model.setObjective(emisiones)

In [25]:
model.write('cgk3_2_END.lp')
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 11 rows, 5 columns and 15 nonzeros
Model fingerprint: 0x5f055b5a
Model has 5 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 5e+00]
  QLMatrix range   [2e+00, 7e+00]
  Objective range  [2e+00, 7e+00]
  QObjective range [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
  QRHS range       [9e+04, 9e+04]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 19 rows, 14 columns, 35 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 8.000e+01
 Factor NZ  : 1.920e+02
 Factor Ops : 2.474e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal        

In [38]:
rows = ["Potencia" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Potencia"+str(t)] = potgen[t].x
units

,0
Potencia0,71.622082
Potencia1,90.000000
Potencia2,68.000000
Potencia3,129.762830
Potencia4,40.615088


In [39]:
rows = ["Costos" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Costos"+str(t)] = a[t]*potgen[t].x*potgen[t].x+b[t]*potgen[t].x+c[t]
units

,0
Costos0,16921.609415
Costos1,34530.170023
Costos2,19888.600022
Costos3,69703.928566
Costos4,7570.904222


In [40]:
rows = ["Emisiones" + str(t) for t in range(g)]
units = pd.DataFrame(columns=range(1), index=rows, data=0.0)

for t in range(g):
            units.loc["Emisiones"+str(t)] = d[t]*potgen[t].x*potgen[t].x+e[t]*potgen[t].x+f[t]
units

,0
Emisiones0,9904.334778
Emisiones1,30566.490021
Emisiones2,23025.730026
Emisiones3,18010.677649
Emisiones4,5582.166245
